# Breast Cancer Detection

This notebook investigates automatic breast cancer detection using classical machine learning models and a GRU–SVM hybrid on the Wisconsin Diagnostic Breast Cancer dataset. We follow a full experimental pipeline: exploratory data analysis, careful data preparation, hyperparameter tuning, evaluation on a held-out test set, and learning-curve analysis to support the paper’s conclusions.

In [ ]:
# Imports des bibliothèques principales pour la manipulation de données et la visualisation
import pandas as pd        # pd : alias standard pour la bibliothèque pandas (tableaux de données type DataFrame)
import numpy as np         # np : alias standard pour NumPy (calcul numérique, tableaux n-dimensionnels)
import matplotlib.pyplot as plt  # plt : interface de tracé principale de Matplotlib
import seaborn as sns      # sns : bibliothèque de visualisation de haut niveau, basée sur Matplotlib

# Outils de scikit-learn pour découper les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

# Outils de prétraitement : standardisation des features et encodage des labels catégoriels
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Modèles linéaires et de type SGD (descente de gradient stochastique)
from sklearn.linear_model import SGDClassifier, SGDRegressor, LogisticRegression

# Support Vector Machines (SVM) pour la classification
from sklearn.svm import SVC

# Réseaux de neurones de type perceptron multicouche (MLP)
from sklearn.neural_network import MLPClassifier

# Méthode des k plus proches voisins (k-NN)
from sklearn.neighbors import KNeighborsClassifier

# Métriques d'évaluation classiques pour la classification et la régression
from sklearn.metrics import (
    confusion_matrix,   # matrice de confusion (TP, TN, FP, FN)
    roc_auc_score,      # aire sous la courbe ROC
    roc_curve,          # points de la courbe ROC (TPR/FPR)
    accuracy_score,     # proportion de prédictions correctes
    recall_score,       # rappel (sensibilité) : proportion de positifs bien détectés
    precision_score,    # précision : proportion de prédictions positives correctes
    f1_score,           # F1-score : moyenne harmonique précision / rappel
    mean_squared_error  # erreur quadratique moyenne (utile pour la régression)
)

# Composants Keras/TensorFlow pour le modèle GRU–SVM
from tensorflow.keras.models import Sequential           # modèle séquentiel (empilement de couches)
from tensorflow.keras.layers import GRU, Dense, Dropout, Input  # couches GRU, denses, dropout et entrée explicite
from tensorflow.keras.callbacks import EarlyStopping      # callback pour arrêter l'entraînement si la val_loss ne s'améliore plus
from tensorflow.keras.optimizers import Adam              # optimiseur Adam (descente de gradient adaptative)
from tensorflow.keras.layers import BatchNormalization    # normalisation de batch pour stabiliser l'entraînement

# Style global des graphiques Seaborn (fond blanc quadrillé pour une meilleure lisibilité)
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
print(f"Dataset shape: {df.shape}")
print(f"Features: {df.shape[1] - 1}")
print(f"Samples: {df.shape[0]}")
df.info()

# 1. Dataset Overview

We first inspect the raw dataset to understand its size, feature set, and basic data types. These descriptive statistics ensure the input is well-formed and provide context (number of patients and features) for the subsequent preprocessing and modeling steps.

In [ ]:
missing_count = df.isnull().sum().sum()
print(f"Total missing values: {missing_count}")

In [ ]:
df.describe()

# 2. Feature Structure

Here we examine the distribution of the target variable (`diagnosis`) and quantify the imbalance between benign and malignant cases. The bar plot and class counts show that benign tumors are more frequent, motivating the later use of SMOTE to balance the training data for fair model comparison.

In [ ]:
plt.figure(figsize=(8, 4))
diagnosis_counts = df['diagnosis'].value_counts()
sns.countplot(x='diagnosis', data=df, hue='diagnosis', legend=False)
plt.title('Diagnosis Distribution')
plt.ylabel('Count')
plt.tight_layout()
plt.show()
print(f"Class distribution:\n{diagnosis_counts}")
print(f"Imbalance ratio: {diagnosis_counts['B'] / diagnosis_counts['M']:.2f}:1 (Benign:Malignant)")

# 3. Data Visualization

We visualize missing values and feature distributions to detect potential quality issues and outliers. The null-value heatmap confirms that the only fully missing column is `Unnamed: 32`, while the per-feature boxplots reveal the range and skewness of each measurement, supporting the decision to standardize features before modeling.

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Null Values Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Samples')
plt.show()

print(f"Total missing values: {df.isnull().sum().sum()}")
print("Missing values per column:")
print(df.isnull().sum()[df.isnull().sum() > 0])

In [ ]:
if 'numeric_cols' not in locals():
    numeric_cols = [col for col in df.columns if col not in ['id', 'Unnamed: 32', 'diagnosis']]

n_features = len(numeric_cols)

n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4))

for i, col in enumerate(numeric_cols):
    ax = plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(y=df[col], ax=ax)
    ax.set_title(f'Distribution de {col}', fontsize=10)
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelsize=8)

plt.tight_layout()
plt.show()

# 4. Correlation Analysis

We compute a correlation matrix between all numeric predictors and the encoded target (`diagnosis_encoded`). The resulting heatmap highlights groups of highly correlated features (e.g., perimeter/area measures) and shows which measurements are most associated with malignancy, guiding later feature reduction and interpretation of model behaviour.

In [ ]:
numeric_cols = [col for col in df.columns if col not in ['id', 'Unnamed: 32', 'diagnosis']]

df_encoded = df.copy()
df_encoded['diagnosis_encoded'] = df['diagnosis'].map({'M': 1, 'B': 0})

corr_cols = numeric_cols + ['diagnosis_encoded']
corr_matrix = df_encoded[corr_cols].corr()

plt.figure(figsize=(20, 18))

sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    square=True,
    cbar_kws={"shrink": 0.8},
    vmin=-1, vmax=1,
    annot_kws={'size': 6},
    linewidths=0.5
)
plt.title('Feature and Target Correlation Matrix', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("High correlations indicate related features. This is expected as features are derived from the same base measurements.")
print("The last row/column shows correlations between features and the target (diagnosis_encoded).")

# 5. Data Preparation

In this step we clean and transform the data before modeling. We drop non-informative and highly redundant columns, encode the target label, stratify the train/test split, standardize all features, and apply SMOTE **only** on the training data to rebalance classes. This pipeline prevents test leakage and ensures that the performance metrics reported later reflect generalization to unseen patients.

In [ ]:
# =====================================================================
# Préparation et découpage du jeu de données
# =====================================================================

import numpy as np  # utilisé ici pour compter les exemples par classe avec np.bincount
from sklearn.model_selection import train_test_split  # fonction pour diviser le jeu de données
from sklearn.preprocessing import LabelEncoder, StandardScaler  # encodage des labels et standardisation des features
from imblearn.over_sampling import SMOTE  # SMOTE : sur-échantillonnage synthétique des minoritaires

# ---------------------------------------------------------------------
# 1) Nettoyage des colonnes non pertinentes ou fortement redondantes
# ---------------------------------------------------------------------
# La colonne 'id' est un identifiant unique sans valeur prédictive directe.
# 'Unnamed: 32' est entièrement vide dans ce jeu de données.
# Les colonnes de périmètre/surface très corrélées sont retirées pour réduire la redondance
# et limiter le risque de sur-apprentissage tout en simplifiant les modèles.
df = df.drop([
    'id',             # identifiant inutile pour la prédiction
    'Unnamed: 32',    # colonne vide
    'perimeter_mean', # fortement corrélée avec radius/area
    'perimeter_se',
    'radius_worst',
    'area_mean',
    'perimeter_worst',
    'area_se',
    'area_worst'
], axis=1, errors='ignore')  # errors='ignore' : ne pas lever d'erreur si une colonne n'existe pas

# ---------------------------------------------------------------------
# 2) Encodage de la variable cible (diagnosis) en 0/1
# ---------------------------------------------------------------------
le = LabelEncoder()                     # instancie un encodeur pour transformer les labels catégoriels en entiers
# 'M' (malignant) et 'B' (benign) deviennent typiquement 1 et 0
# fit_transform apprend le mapping puis applique la transformation sur la colonne 'diagnosis'
df['diagnosis'] = le.fit_transform(df['diagnosis'])

# ---------------------------------------------------------------------
# 3) Séparation features / cible
# ---------------------------------------------------------------------
X = df.drop('diagnosis', axis=1)  # X : toutes les colonnes sauf la cible
y = df['diagnosis']               # y : variable cible binaire (0 = bénin, 1 = malin)

# ---------------------------------------------------------------------
# 4) Split entraînement / test (avec stratification)
# ---------------------------------------------------------------------
# test_size=0.3 : 30 % des données pour le test, 70 % pour l'entraînement
# random_state=42 : graine fixe pour rendre le split reproductible
# stratify=y : conserve le même ratio de classes (0/1) dans train et test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Affichage de la taille des ensembles pour vérifier le split
print(f"Train set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Distribution des classes AVANT SMOTE (important pour justifier le rééquilibrage)
print("\nTrain class distribution (before SMOTE):")
print(f"  Benign (0): {np.bincount(y_train)[0]}")   # nombre de cas bénins dans l'entraînement
print(f"  Malignant (1): {np.bincount(y_train)[1]}")  # nombre de cas malins

print("\nTest class distribution:")
print(f"  Benign (0): {np.bincount(y_test)[0]}")
print(f"  Malignant (1): {np.bincount(y_test)[1]}")

# ---------------------------------------------------------------------
# 5) Standardisation des caractéristiques
# ---------------------------------------------------------------------
# StandardScaler met chaque feature à moyenne 0 et écart-type 1, ce qui est crucial
# pour les modèles sensibles à l'échelle (SVM, MLP, k-NN, etc.).
scaler = StandardScaler()

# fit_transform sur X_train : apprend les moyennes/écarts-types puis transforme
X_train_scaled = scaler.fit_transform(X_train)
# transform sur X_test : applique EXACTEMENT la même transformation (pas de fuite d'info)
X_test_scaled = scaler.transform(X_test)

print("\nStandardization applied: mean = 0, std = 1")

# ---------------------------------------------------------------------
# 6) Rééquilibrage des classes avec SMOTE sur l'entraînement
# ---------------------------------------------------------------------
# SMOTE génère de nouveaux exemples synthétiques de la classe minoritaire (ici les malins)
# en interpolant entre voisins proches, afin d'obtenir un jeu de train équilibré.
smote = SMOTE(random_state=42)  # random_state=42 pour rendre l'échantillonnage reproductible

# On applique SMOTE UNIQUEMENT sur X_train_scaled, y_train (jamais sur le test)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("\nTrain class distribution (after SMOTE):")
print(f"  Benign (0): {np.bincount(y_train_smote)[0]}")
print(f"  Malignant (1): {np.bincount(y_train_smote)[1]}")

# ---------------------------------------------------------------------
# 7) Split supplémentaire pour la recherche d'hyperparamètres
# ---------------------------------------------------------------------
# Pour la validation (tuning), on découpe de nouveau X_train_scaled en sous-ensemble d'entraînement
# et de validation, afin d'éviter toute fuite d'information en utilisant le test pour le tuning.
# test_size=0.3 : 30 % de l'ancien train devient validation.
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train_scaled,
    y_train,
    test_size=0.3,
    random_state=42
)

# On crée une deuxième instance de SMOTE pour rééquilibrer le sous-ensemble utilisé pour le tuning
smote_tune = SMOTE(random_state=42)
X_train_sub_smote, y_train_sub_smote = smote_tune.fit_resample(X_train_sub, y_train_sub)

# 6. Modeling

We compare a set of classical and neural models under a unified experimental protocol. For each model we perform hyperparameter tuning on a validation split (`X_train_sub`, `X_val`), then retrain the best configuration on the SMOTE-balanced training data (`X_train_smote`) before evaluating on the held-out test set. This section defines the candidate models that will later be compared via metrics, confusion matrices, and learning curves.

## 6.1 Linear Regression (SGDRegressor baseline)

We use `SGDRegressor` with a squared-error loss as a simple linear baseline model. Its continuous outputs are thresholded at 0.5 to obtain class labels and rescaled with `MinMaxScaler` to approximate probabilities for ROC–AUC computation. Learning rate and number of iterations are selected to maximize validation AUC, after which the best model is retrained on the SMOTE-balanced training set.

In [ ]:
# =====================================================================
# 6.1 Régression linéaire (baseline avec SGDRegressor)
# =====================================================================

# Liste des taux d'apprentissage candidats pour la descente de gradient stochastique.
# 1e-4, 1e-3, 1e-2 : on explore trois ordres de grandeur pour contrôler la taille des pas de mise à jour.
learning_rates = [1e-4, 1e-3, 1e-2]

# Liste des nombres maximum d'itérations (passes sur les données) testés.
# 500, 1000, 2000 : plus le nombre est grand, plus le modèle peut converger finement, mais avec un risque de sur-apprentissage.
iterations = [500, 1000, 2000]

# Meilleure valeur d'AUC observée jusqu'à présent.
# On l'initialise à -inf pour être sûr que la première configuration testée sera retenue.
best_auc_lr = -np.inf

# Hyperparamètres optimaux (taux d'apprentissage et nombre d'itérations) trouvés pendant la recherche.
best_eta = None
best_iter = None

# ---------------------------------------------------------------------
# Boucle de recherche d'hyperparamètres sur (eta0, max_iter)
# ---------------------------------------------------------------------
for eta in learning_rates:           # eta : taux d'apprentissage initial (eta0) testé
    for it in iterations:            # it : nombre maximum d'itérations testé
        # Création d'un modèle de régression linéaire entraîné par SGD
        linear_model = SGDRegressor(
            loss='squared_error',     # fonction de perte : erreur quadratique moyenne (régression linéaire)
            learning_rate='constant', # taux d'apprentissage constant (utilise eta0 à chaque itération)
            eta0=eta,                 # taux d'apprentissage testé dans la boucle
            max_iter=it,              # nombre maximal de passes sur les données
            random_state=42           # graine pour rendre l'entraînement reproductible
        )
        # Entraînement sur le sous-ensemble de train utilisé pour le tuning (sans SMOTE global)
        linear_model.fit(X_train_sub, y_train_sub)

        # Prédiction continue (valeurs réelles entre -inf et +inf) sur la validation
        linear_y_val_continuous = linear_model.predict(X_val)

        # Seuil à 0,5 pour obtenir des prédictions de classe (0 ou 1) à partir des sorties continues
        lr_y_val_pred = (linear_y_val_continuous >= 0.5).astype(int)

        # Import local de MinMaxScaler pour transformer les sorties continues en pseudo‑probabilités dans [0, 1]
        from sklearn.preprocessing import MinMaxScaler
        prob_scaler = MinMaxScaler()  # normalise les valeurs continues sur l'intervalle [0, 1]

        # On ajuste le scaler sur les prédictions d'entraînement pour apprendre le min/max des scores
        linear_y_train_continuous = linear_model.predict(X_train_sub)
        prob_scaler.fit(linear_y_train_continuous.reshape(-1, 1))

        # On transforme ensuite les scores de validation en pseudo‑probabilités entre 0 et 1
        lr_y_val_proba = prob_scaler.transform(
            linear_y_val_continuous.reshape(-1, 1)
        ).flatten()

        # Calcul de l'AUC ROC sur le jeu de validation pour cette configuration (eta, it)
        auc = roc_auc_score(y_val, lr_y_val_proba)

        # Si l'AUC est meilleure que la meilleure actuelle, on met à jour les hyperparamètres optimaux
        if auc > best_auc_lr:
            best_auc_lr = auc   # nouvelle meilleure AUC
            best_eta = eta      # meilleur taux d'apprentissage trouvé
            best_iter = it      # meilleur nombre d'itérations trouvé

# ---------------------------------------------------------------------
# Ré‑entraînement du meilleur modèle sur l'ensemble d'entraînement SMOTE
# ---------------------------------------------------------------------
linear_model = SGDRegressor(
    loss='squared_error',        # même type de perte que pendant le tuning
    learning_rate='constant',    # même stratégie de taux d'apprentissage
    eta0=best_eta,               # taux d'apprentissage optimal trouvé
    max_iter=best_iter,          # nombre d'itérations optimal trouvé
    random_state=42
)

# Entraînement final sur les données d'entraînement rééquilibrées (X_train_smote, y_train_smote)
linear_model.fit(X_train_smote, y_train_smote)

# Prédictions continues sur le jeu de test (valeurs réelles)
linear_y_test_continuous = linear_model.predict(X_test_scaled)

# Conversion en prédictions de classe binaires par seuillage à 0,5
lr_y_test_pred = (linear_y_test_continuous >= 0.5).astype(int)

# Re‑ajustement d'un MinMaxScaler sur les scores continus d'entraînement SMOTE
prob_scaler = MinMaxScaler()
linear_y_train_continuous = linear_model.predict(X_train_smote)
prob_scaler.fit(linear_y_train_continuous.reshape(-1, 1))

# Transformation des scores de test en pseudo‑probabilités dans [0, 1]
lr_y_test_proba = prob_scaler.transform(
    linear_y_test_continuous.reshape(-1, 1)
).flatten()

## 6.2 Logistic Regression (Softmax classifier)

Logistic regression serves as a strong linear probabilistic classifier and is a key reference model in the paper. We tune the regularization strength `C`, solver, and maximum iterations on the validation set using ROC–AUC, then retrain the best configuration on `X_train_smote`. The resulting model provides well-calibrated probabilities and a clear decision boundary, against which more complex models are compared.

In [ ]:
# =====================================================================
# 6.2 Régression logistique (classifieur Softmax)
# =====================================================================

# Liste de valeurs candidates pour le paramètre de régularisation C.
# C contrôle la pénalisation des poids : petites valeurs = forte régularisation, grandes valeurs = modèle plus flexible.
C_values = [0.1, 1, 10]

# Liste des solveurs pris en charge par scikit-learn pour la régression logistique binaire.
# - 'lbfgs' : quasi-Newton, efficace pour des jeux de données de taille moyenne
# - 'liblinear' : utilise la bibliothèque LIBLINEAR, stable pour des jeux de données petits/moyens
# - 'newton-cg' : variante de Newton conjugué, adaptée au softmax
solvers = ['lbfgs', 'liblinear', 'newton-cg']

# Nombres maximum d'itérations testés pour s'assurer de la convergence.
# Plus max_iter est grand, plus l'optimisation peut converger, au prix d'un temps d'entraînement plus long.
iterations = [1000, 3000, 5000]

# Meilleure AUC observée jusqu'à présent pour la régression logistique.
best_auc_softmax = -np.inf

# Hyperparamètres optimaux retenus : C, solveur et max_iter.
best_softmax_C = None
best_softmax_solver = None
best_softmax_max_iter = None

# ---------------------------------------------------------------------
# Boucle de recherche d'hyperparamètres (C, solver, max_iter)
# ---------------------------------------------------------------------
for C_val in C_values:            # C_val : valeur candidate de C
    for solver in solvers:        # solver : type d'algorithme d'optimisation utilisé
        for it in iterations:     # it : nombre maximal d'itérations
            # Instanciation d'un modèle de régression logistique
            softmax_model = LogisticRegression(
                C=C_val,             # intensité de la régularisation (inverse de lambda)
                solver=solver,       # solveur utilisé pour optimiser la log‑vraisemblance
                max_iter=it,         # borne supérieure sur le nombre d'itérations d'optimisation
                random_state=42      # graine pour la reproductibilité
            )
            # Entraînement sur le sous-ensemble de train pour le tuning
            softmax_model.fit(X_train_sub, y_train_sub)

            # Probabilités prédites pour la classe positive (maligne = 1) sur la validation
            softmax_y_val_proba = softmax_model.predict_proba(X_val)[:, 1]

            # Calcul de l'AUC ROC sur le jeu de validation pour évaluer cette configuration
            auc = roc_auc_score(y_val, softmax_y_val_proba)

            # Mise à jour des meilleurs hyperparamètres si l'AUC est améliorée
            if auc > best_auc_softmax:
                best_auc_softmax = auc       # nouvelle meilleure AUC
                best_softmax_C = C_val       # meilleure valeur de C trouvée
                best_softmax_solver = solver # meilleur solveur trouvé
                best_softmax_max_iter = it   # meilleur nombre d'itérations trouvé

# ---------------------------------------------------------------------
# Ré‑entraînement du meilleur modèle de régression logistique sur SMOTE
# ---------------------------------------------------------------------
softmax_model = LogisticRegression(
    C=best_softmax_C,                # C optimal
    solver=best_softmax_solver,      # solveur optimal
    max_iter=best_softmax_max_iter,  # nombre d'itérations optimal
    random_state=42
)

# Entraînement final sur les données d'entraînement rééquilibrées
softmax_model.fit(X_train_smote, y_train_smote)

# Probabilités prévues pour la classe positive sur le jeu de test
softmax_y_test_proba = softmax_model.predict_proba(X_test_scaled)[:, 1]

# Prédictions de classe (0/1) sur le test en utilisant le seuil par défaut de scikit-learn (0,5)
softmax_y_test_pred = softmax_model.predict(X_test_scaled)

## 6.3 MLP (feedforward neural network)

This model is a fully connected neural network that can capture non-linear relationships between features and malignancy. We perform a grid search over the number and size of hidden layers, weight decay (`alpha`), learning rate, and maximum epochs, selecting the configuration that maximizes validation ROC–AUC. The chosen MLP is then retrained on the balanced training set and evaluated on the test set alongside the other models.

In [ ]:
# =====================================================================
# 6.3 MLP (réseau de neurones fully-connected)
# =====================================================================

# Taux d'apprentissage initiaux candidats pour l'optimiseur interne du MLP.
learning_rates = [1e-4, 1e-3]

# Nombres d'itérations maximales (époques) à tester pour l'entraînement du MLP.
iterations = [1000, 3000, 5000]

# Architectures candidates pour les couches cachées :
# - (200, 200) : 2 couches de 200 neurones
# - (500, 500, 500) : 3 couches plus larges de 500 neurones
# - (300, 300, 300) : 3 couches intermédiaires de 300 neurones
hidden_layers = [(200, 200), (500, 500, 500), (300, 300, 300)]

# Valeurs candidates de la régularisation L2 (weight decay) via le paramètre alpha.
# Plus alpha est grand, plus les poids sont pénalisés (modèle plus simple, moins de sur-apprentissage).
alphas = [1e-4, 1e-3, 1e-2, 1e-1]

# Suivi de la meilleure AUC obtenue pendant la recherche d'hyperparamètres.
best_auc_mlp = -np.inf

# Hyperparamètres optimaux retenus pour le MLP.
best_mlp_hidden = None
best_mlp_alpha = None
best_mlp_lr = None
best_mlp_it = None

# ---------------------------------------------------------------------
# Boucle de recherche en grille sur (architecture, alpha, lr, max_iter)
# ---------------------------------------------------------------------
for hls in hidden_layers:          # hls : tuple décrivant la taille de chaque couche cachée
    for alpha in alphas:           # alpha : coefficient de régularisation L2
        for lr in learning_rates:  # lr : learning_rate_init
            for it in iterations:  # it : nombre maximal d'itérations
                # Instanciation du MLP avec un ensemble donné d'hyperparamètres
                mlp_model = MLPClassifier(
                    hidden_layer_sizes=hls,        # architecture des couches cachées testée
                    learning_rate_init=lr,         # taux d'apprentissage initial
                    learning_rate='adaptive',      # le taux s'adapte en fonction de la progression de l'erreur
                    alpha=alpha,                   # pénalisation L2 des poids
                    max_iter=it,                   # nombre maximal d'itérations
                    batch_size=128,                # taille du mini-batch (stabilise les mises à jour)
                    n_iter_no_change=20,           # nombre d'itérations sans amélioration avant arrêt anticipé
                    early_stopping=True,           # active l'arrêt anticipé basé sur la validation interne
                    random_state=42,               # graine pour reproductibilité
                    validation_fraction=0.1        # part de l'entraînement utilisée comme validation interne
                )

                # Entraînement du MLP sur le sous-ensemble de train pour le tuning
                mlp_model.fit(X_train_sub, y_train_sub)

                # Probabilités pour la classe positive sur le jeu de validation externe X_val
                mlp_y_val_proba = mlp_model.predict_proba(X_val)[:, 1]

                # Calcul de l'AUC ROC pour cette configuration d'hyperparamètres
                auc = roc_auc_score(y_val, mlp_y_val_proba)

                # Mise à jour des meilleurs hyperparamètres si l'AUC est améliorée
                if auc > best_auc_mlp:
                    best_auc_mlp = auc
                    best_mlp_hidden = hls
                    best_mlp_alpha = alpha
                    best_mlp_lr = lr
                    best_mlp_it = it

# ---------------------------------------------------------------------
# Ré‑entraînement du MLP avec les meilleurs hyperparamètres sur SMOTE
# ---------------------------------------------------------------------
mlp_model = MLPClassifier(
    hidden_layer_sizes=best_mlp_hidden,  # architecture optimale
    learning_rate_init=best_mlp_lr,      # taux d'apprentissage optimal
    alpha=best_mlp_alpha,                # régularisation L2 optimale
    max_iter=best_mlp_it,                # nombre d'itérations optimal
    early_stopping=True,                 # arrêt anticipé basé sur validation interne
    random_state=42,
    verbose=0,                           # verbose=0 : ne pas afficher les logs d'entraînement
    validation_fraction=0.1
)

# Entraînement final sur les données d'entraînement rééquilibrées
mlp_model.fit(X_train_smote, y_train_smote)

# Probabilités pour la classe positive sur le jeu de test
mlp_y_test_proba = mlp_model.predict_proba(X_test_scaled)[:, 1]

# Prédictions de classe (0/1) sur le jeu de test
mlp_y_test_pred = mlp_model.predict(X_test_scaled)

## 6.4 L2-SVM (linear support vector machine)

We train a linear SVM with probabilistic outputs to obtain a maximum-margin classifier in the original feature space. The regularization parameter `C` is tuned on the validation set using ROC–AUC, and the best model is refit on `X_train_smote`. This provides a strong linear margin-based baseline that is often competitive with neural approaches on tabular biomedical data.

In [ ]:
# =====================================================================
# 6.4 L2-SVM (machine à vecteurs de support linéaire)
# =====================================================================

# Valeurs candidates pour le paramètre de régularisation C de la SVM linéaire.
# C élevé -> marge plus petite mais moins d'erreurs d'entraînement ; C faible -> marge plus large mais plus d'erreurs permises.
C_values = [0.1, 1, 10]

# Meilleure AUC obtenue pour la SVM sur la validation.
best_auc_svm = -np.inf

# Meilleure valeur de C retenue.
best_svm_C = None

# ---------------------------------------------------------------------
# Boucle de recherche sur C pour la SVM linéaire
# ---------------------------------------------------------------------
for C in C_values:  # C : intensité de la pénalisation des erreurs
    svm_model = SVC(
        C=C,                  # valeur de C testée
        kernel='linear',      # noyau linéaire : frontière de décision planaire
        probability=True,     # active la calibration des probabilités via Platt scaling
        random_state=42       # graine pour reproductibilité (utile surtout pour le calibrage)
    )
    # Entraînement de la SVM sur le sous-ensemble d'entraînement pour le tuning
    svm_model.fit(X_train_sub, y_train_sub)

    # Probabilités prévues pour la classe positive sur la validation
    svm_y_val_proba = svm_model.predict_proba(X_val)[:, 1]

    # Calcul de l'AUC ROC pour évaluer la qualité de la séparation des classes
    auc = roc_auc_score(y_val, svm_y_val_proba)

    # Mise à jour de la meilleure valeur de C si l'AUC est améliorée
    if auc > best_auc_svm:
        best_auc_svm = auc
        best_svm_C = C

# ---------------------------------------------------------------------
# Ré‑entraînement de la SVM linéaire avec le meilleur C sur SMOTE
# ---------------------------------------------------------------------
svm_model = SVC(
    C=best_svm_C,          # meilleur paramètre C trouvé
    kernel='linear',       # noyau linéaire
    probability=True,
    random_state=42
)

# Entraînement final sur les données d'entraînement rééquilibrées
svm_model.fit(X_train_smote, y_train_smote)

# Prédictions de classe sur le jeu de test
svm_y_test_pred = svm_model.predict(X_test_scaled)

# Probabilités prévues pour la classe positive sur le jeu de test
svm_y_test_proba = svm_model.predict_proba(X_test_scaled)[:, 1]

## 6.5 k-Nearest Neighbours (L1 and L2)

We evaluate instance-based learners using k-nearest neighbours with both Manhattan (L1) and Euclidean (L2) distances. For each metric we select the optimal number of neighbours `k` by maximizing validation ROC–AUC, then retrain the best L1-NN and L2-NN on the balanced training set. These models provide a non-parametric baseline that can adapt to local structure in the feature space.

In [ ]:
# =====================================================================
# 6.5 k plus proches voisins (L1 et L2)
# =====================================================================

# Ensemble de valeurs k candidates (nombre de voisins) pour les modèles k-NN.
# De petits k (1,3) capturent des structures très locales mais sont plus sensibles au bruit.
# Des k plus grands (7,9) lissent davantage la décision.
k_values = [1, 3, 5, 7, 9]

# Meilleure AUC pour le modèle k-NN avec distance L1 (Manhattan).
best_auc_knn_l1 = -np.inf
best_knn_l1_k = None  # valeur optimale de k pour L1

# Meilleure AUC pour le modèle k-NN avec distance L2 (euclidienne).
best_auc_knn_l2 = -np.inf
best_knn_l2_k = None  # valeur optimale de k pour L2

# ---------------------------------------------------------------------
# Boucle sur les différentes valeurs de k pour L1 et L2
# ---------------------------------------------------------------------
for k in k_values:
    # ------- k-NN avec distance de Manhattan (L1) -------
    knn_l1 = KNeighborsClassifier(
        n_neighbors=k,        # nombre de voisins pris en compte
        metric='minkowski',   # métrique Minkowski générale
        p=1                   # p=1 => distance de Manhattan (L1)
    )
    knn_l1.fit(X_train_sub, y_train_sub)  # entraînement sur le sous-ensemble d'entraînement

    # Probabilités pour la classe positive sur la validation
    knn_l1_y_val_proba = knn_l1.predict_proba(X_val)[:, 1]

    # AUC ROC pour ce k avec distance L1
    auc = roc_auc_score(y_val, knn_l1_y_val_proba)
    if auc > best_auc_knn_l1:
        best_auc_knn_l1 = auc
        best_knn_l1_k = k

    # ------- k-NN avec distance euclidienne (L2) -------
    knn_l2 = KNeighborsClassifier(
        n_neighbors=k,        # même k testé
        metric='minkowski',   # métrique Minkowski
        p=2                   # p=2 => distance euclidienne (L2)
    )
    knn_l2.fit(X_train_sub, y_train_sub)

    # Probabilités pour la classe positive sur la validation
    knn_l2_y_val_proba = knn_l2.predict_proba(X_val)[:, 1]

    # AUC ROC pour ce k avec distance L2
    auc = roc_auc_score(y_val, knn_l2_y_val_proba)
    if auc > best_auc_knn_l2:
        best_auc_knn_l2 = auc
        best_knn_l2_k = k

# ---------------------------------------------------------------------
# Ré‑entraînement des meilleurs modèles k-NN (L1 et L2) sur SMOTE
# ---------------------------------------------------------------------
# Modèle k-NN L1 avec le meilleur k trouvé.
knn_l1 = KNeighborsClassifier(
    n_neighbors=best_knn_l1_k,
    metric='minkowski',
    p=1
)
knn_l1.fit(X_train_smote, y_train_smote)

# Prédictions de classe et probabilités sur le jeu de test
knn_l1_y_test_pred = knn_l1.predict(X_test_scaled)
knn_l1_y_test_proba = knn_l1.predict_proba(X_test_scaled)[:, 1]

# Modèle k-NN L2 avec le meilleur k trouvé.
knn_l2 = KNeighborsClassifier(
    n_neighbors=best_knn_l2_k,
    metric='minkowski',
    p=2
)
knn_l2.fit(X_train_smote, y_train_smote)

# Prédictions de classe et probabilités sur le jeu de test
knn_l2_y_test_pred = knn_l2.predict(X_test_scaled)
knn_l2_y_test_proba = knn_l2.predict_proba(X_test_scaled)[:, 1]

## 6.6 GRU–SVM hybrid

Finally, we build a hybrid model that learns a non-linear representation with a GRU network and then applies an RBF-kernel SVM on the extracted features. The GRU is trained with early stopping on a reshaped version of the tabular data, and its penultimate layer is used as a compact feature embedding. We then tune the SVM regularization parameter `C` on these embeddings and evaluate the resulting GRU–SVM on the test set, providing a deep learning–based point of comparison with the classical models.

In [ ]:
# =====================================================================
# 6.6 Hybride GRU–SVM
# =====================================================================

# Valeurs candidates de C pour la SVM à noyau RBF appliquée sur les features extraites par le GRU.
C_values = [0.1, 1, 5]

# Meilleure AUC obtenue pour la combinaison GRU + SVM.
best_auc_gru = -np.inf

# Hyperparamètres/infos retenus pour le meilleur modèle GRU–SVM.
best_gru_units = None   # nombre de neurones GRU (fixé à 64 ici, mais conservé pour la cohérence)
best_gru_svm_C = None   # meilleure valeur de C pour la SVM

# ---------------------------------------------------------------------
# Préparation des entrées pour le GRU
# ---------------------------------------------------------------------
# Nombre de caractéristiques d'entrée (dimensions des vecteurs de features tabulaires).
n_features = X_train_sub_smote.shape[1]

# Reshape des données tabulaires en séquences de longueur n_features avec 1 feature par "time-step".
# Forme attendue par le GRU : (n_samples, timesteps, features_par_timestep)
X_train_gru = X_train_sub_smote.reshape(X_train_sub_smote.shape[0], n_features, 1)
X_val_gru = X_val.reshape(X_val.shape[0], n_features, 1)
X_test_gru = X_test_scaled.reshape(X_test_scaled.shape[0], n_features, 1)

# ---------------------------------------------------------------------
# Définition de l'architecture GRU
# ---------------------------------------------------------------------
gru_model = Sequential([
    # Couche d'entrée explicitant la forme des séquences d'entrée
    Input(shape=(n_features, 1)),

    # Couche GRU avec 64 unités : résume la séquence en un vecteur de taille 64
    GRU(64, return_sequences=False),

    # Dropout 0.3 : éteint aléatoirement 30 % des unités pour limiter le sur-apprentissage
    Dropout(0.3),  # réduit par rapport à 0.6 pour garder plus de capacité

    # Couche dense intermédiaire de 32 neurones avec activation ReLU pour plus de non‑linéarité
    Dense(32, activation='relu'),

    # BatchNormalization : normalise les activations pour stabiliser et accélérer l'entraînement
    BatchNormalization(),

    # Couche de sortie de taille 1 avec sigmoïde : renvoie une probabilité de malignité entre 0 et 1
    Dense(1, activation='sigmoid')
])

# Compilation du modèle GRU
# - Adam : optimiseur adaptatif bien adapté aux réseaux profonds
# - binary_crossentropy : perte standard pour la classification binaire
# - metrics=['accuracy'] : suivre l'accuracy pendant l'entraînement
gru_model.compile(
    optimizer=Adam(learning_rate=1e-3),  # taux d'apprentissage 1e-3, valeur classique stable
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# EarlyStopping : arrête l'entraînement si la perte de validation ne s'améliore plus.
early_stopping = EarlyStopping(
    monitor='val_loss',       # on surveille la perte de validation
    patience=30,              # on attend 30 époques sans amélioration avant d'arrêter
    restore_best_weights=True,# on restaure les poids correspondant à la meilleure val_loss
    verbose=0
)

# Entraînement du GRU sur les données séquentielles dérivées de X_train_sub_smote
# - epochs=500 : nombre maximal d'époques (l'early stopping arrêtera plus tôt si nécessaire)
# - batch_size=128 : taille du mini-batch
# - validation_split=0.2 : 20 % des données d'entraînement utilisés comme validation interne
# - callbacks=[early_stopping] : applique l'arrêt anticipé décrit ci‑dessus
gru_model.fit(
    X_train_gru, y_train_sub_smote,
    epochs=500,
    batch_size=128,
    validation_split=0.2,
    verbose=0,
    callbacks=[early_stopping]
)

# ---------------------------------------------------------------------
# Extraction des caractéristiques apprises par le GRU
# ---------------------------------------------------------------------
from tensorflow.keras.models import Model

# On définit un modèle intermédiaire qui prend la même entrée que le GRU
# mais dont la sortie est la couche dense de 32 neurones (avant la normalisation batch et la sortie sigmoïde).
model_input = gru_model.layers[0].input
feature_extractor = Model(
    inputs=model_input,
    outputs=gru_model.layers[-3].output  # couche Dense(32, activation='relu')
)

# Extraction des features pour train / val / test (sorties 32D)
gru_train_features = feature_extractor.predict(X_train_gru, verbose=0)
gru_val_features = feature_extractor.predict(X_val_gru, verbose=0)
gru_test_features = feature_extractor.predict(X_test_gru, verbose=0)

# ---------------------------------------------------------------------
# Recherche du meilleur C pour la SVM RBF sur les features extraites
# ---------------------------------------------------------------------
for C_val in C_values:
    gru_svm_model = SVC(
        kernel='rbf',      # noyau RBF pour capturer des frontières non linéaires dans l'espace des embeddings
        C=C_val,           # valeur de C testée
        probability=True   # permet d'obtenir des probabilités calibrées
    )
    gru_svm_model.fit(gru_train_features, y_train_sub_smote)

    # Probabilités pour la classe positive sur les embeddings de validation
    gru_svm_y_val_proba = gru_svm_model.predict_proba(gru_val_features)[:, 1]

    # AUC ROC pour cette valeur de C
    auc = roc_auc_score(y_val, gru_svm_y_val_proba)
    if auc > best_auc_gru:
        best_auc_gru = auc
        best_gru_units = 64      # taille de la couche GRU (fixe dans ce notebook)
        best_gru_svm_C = C_val   # meilleure valeur de C pour la SVM

# ---------------------------------------------------------------------
# Entraînement final de la SVM RBF GRU–SVM sur l'ensemble train complet
# ---------------------------------------------------------------------
gru_svm_model = SVC(
    kernel='rbf',
    C=best_gru_svm_C,      # meilleur C trouvé
    probability=True,
    random_state=42
)

# Entraînement de la SVM sur les features GRU extraites du train complet
gru_svm_model.fit(gru_train_features, y_train_sub_smote)

# Prédictions de classe et probabilités sur le jeu de test (dans l'espace des embeddings GRU)
gru_svm_y_test_pred = gru_svm_model.predict(gru_test_features)
gru_svm_y_test_proba = gru_svm_model.predict_proba(gru_test_features)[:, 1]

# 7. Evaluation

We now evaluate all tuned models on the held-out test set using a comprehensive set of metrics (accuracy, ROC–AUC, precision, recall, F1-score, specificity, FPR/FNR, TPR). The resulting table (`results_df`) summarizes global performance and supports the ranking of models reported in the conclusion, while the printed hyperparameters document the exact configurations used.

In [ ]:
# =====================================================================
# 7. Évaluation : calcul des métriques sur le jeu de test
# =====================================================================

from sklearn.metrics import (
    confusion_matrix, accuracy_score, roc_auc_score,
    recall_score, precision_score, f1_score
)
import pandas as pd

# ---------------------------------------------------------------------
# Fonction utilitaire pour calculer un ensemble complet de métriques
# ---------------------------------------------------------------------
def calculate_metrics(y_true, y_pred, y_proba):
    """Calcule plusieurs métriques de classification binaire à partir
    des prédictions discrètes (y_pred) et probabilistes (y_proba)."""

    # Matrice de confusion 2x2 :
    # [[TN, FP],
    #  [FN, TP]]
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    return {
        # Accuracy : proportion globale de prédictions correctes
        'accuracy': accuracy_score(y_true, y_pred),

        # ROC-AUC : aire sous la courbe ROC (sensibilité vs 1-spécificité)
        'roc_auc': roc_auc_score(y_true, y_proba),

        # Recall (sensibilité) : TP / (TP + FN), proportion de malins correctement détectés
        'recall': recall_score(y_true, y_pred, pos_label=1),

        # Précision : TP / (TP + FP), proportion de prédictions "malin" correctes
        'precision': precision_score(y_true, y_pred, pos_label=1),

        # F1-score : moyenne harmonique précision / rappel
        'f1_score': f1_score(y_true, y_pred, pos_label=1),

        # Spécificité : TN / (TN + FP), capacité à reconnaître les cas bénins
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0.0,

        # FPR (False Positive Rate) : FP / (FP + TN)
        'FPR': fp / (fp + tn) if (fp + tn) > 0 else 0.0,

        # FNR (False Negative Rate) : FN / (FN + TP)
        'FNR': fn / (fn + tp) if (fn + tp) > 0 else 0.0,

        # TPR (True Positive Rate) : identique au recall ici
        'TPR': tp / (tp + fn) if (tp + fn) > 0 else 0.0
    }

# ---------------------------------------------------------------------
# Calcul des métriques pour chaque modèle sur le jeu de test
# ---------------------------------------------------------------------
test_results = {
    'Linear Regression': calculate_metrics(y_test, lr_y_test_pred, lr_y_test_proba),
    'Softmax Regression': calculate_metrics(y_test, softmax_y_test_pred, softmax_y_test_proba),
    'MLP': calculate_metrics(y_test, mlp_y_test_pred, mlp_y_test_proba),
    'L2-SVM': calculate_metrics(y_test, svm_y_test_pred, svm_y_test_proba),
    'L1-NN': calculate_metrics(y_test, knn_l1_y_test_pred, knn_l1_y_test_proba),
    'L2-NN': calculate_metrics(y_test, knn_l2_y_test_pred, knn_l2_y_test_proba),
    'GRU-SVM': calculate_metrics(y_test, gru_svm_y_test_pred, gru_svm_y_test_proba)
}

# Conversion du dictionnaire de résultats en DataFrame pour un affichage tabulaire
results_df = pd.DataFrame(test_results).T  # .T : transpose pour avoir un modèle par ligne
print(results_df)

# ---------------------------------------------------------------------
# Récapitulatif des meilleurs hyperparamètres utilisés pour chaque modèle
# ---------------------------------------------------------------------
# Ce dictionnaire permet de documenter précisément les configurations
# des modèles qui ont conduit aux résultats reportés.
best_hyperparameters = {
    "Linear Regression": {
        "eta0": best_eta,              # meilleur taux d'apprentissage pour SGDRegressor
        "max_iter": best_iter          # meilleur nombre d'itérations pour SGDRegressor
    },
    "Softmax Regression": {
        "C": best_softmax_C,           # meilleur C pour la régression logistique
        "solver": best_softmax_solver  # meilleur solveur pour la régression logistique
    },
    "MLP": {
        "hidden_layer_sizes": best_mlp_hidden,  # meilleure architecture cachée
        "alpha": best_mlp_alpha                 # meilleur poids de régularisation L2
    },
    "L2-SVM": {
        "C": best_svm_C,               # meilleur C pour la SVM linéaire
        "kernel": "linear"            # noyau utilisé
    },
    "L1-NN": {
        "k": best_knn_l1_k,           # meilleur nombre de voisins pour k-NN L1
        "metric": "manhattan"        # distance de Manhattan (L1)
    },
    "L2-NN": {
        "k": best_knn_l2_k,           # meilleur nombre de voisins pour k-NN L2
        "metric": "euclidean"        # distance euclidienne (L2)
    },
    "GRU-SVM": {
        "GRU_units": best_gru_units,  # taille de la couche GRU
        "SVM_C": best_gru_svm_C       # meilleur C pour la SVM RBF
    }
}

# Affichage lisible des hyperparamètres retenus pour chaque modèle
for model, params in best_hyperparameters.items():
    print(f"{model}:")
    for p, v in params.items():
        print(f"  - {p}: {v}")
    print()

# 8. Confusion Matrices

To complement the aggregate metrics, we plot the confusion matrix of each model on the test set. These heatmaps reveal the balance between true positives, true negatives, false positives and especially false negatives, which are critical in cancer detection. Together with the metric table, they help assess which models best limit missed malignant cases while keeping the false-alarm rate acceptable.

In [ ]:
# =====================================================================
# 8. Matrices de confusion : visualisation détaillée des erreurs
# =====================================================================

# Liste des modèles et de leurs prédictions de classe sur le jeu de test.
# Chaque tuple contient : (nom lisible, vecteur y_pred correspondant).
models_data = [
    ('Linear Regression', lr_y_test_pred),
    ('Softmax Regression', softmax_y_test_pred),
    ('MLP', mlp_y_test_pred),
    ('L2-SVM', svm_y_test_pred),
    ('L1-NN', knn_l1_y_test_pred),
    ('L2-NN', knn_l2_y_test_pred),
    ('GRU-SVM', gru_svm_y_test_pred)
]

# Nombre total de modèles à afficher.
n_models = len(models_data)

# Disposition de la grille de sous-graphiques : 3 colonnes, n_rows lignes.
n_cols = 3
n_rows = (n_models + n_cols - 1) // n_cols  # division plafond pour couvrir tous les modèles

# Création de la figure et des axes pour les heatmaps de matrices de confusion.
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4*n_rows))

# Gestion du cas où il n'y a qu'une seule ligne : axes peut être un seul axe ou un tableau.
if n_rows == 1:
    axes = axes if isinstance(axes, np.ndarray) else [axes]
else:
    axes = axes.flatten()  # mise à plat pour indexer simplement avec idx

# Boucle sur chaque modèle pour tracer sa matrice de confusion.
for idx, (name, y_pred) in enumerate(models_data):
    ax = axes[idx]  # axe courant

    # Matrice de confusion entre les vraies étiquettes y_test et les prédictions y_pred
    cm = confusion_matrix(y_test, y_pred)

    # Heatmap Seaborn de la matrice de confusion
    sns.heatmap(
        cm,
        annot=True,                    # affiche les nombres dans les cases
        fmt='d',                       # format entier
        cmap='Blues',                  # colormap bleue
        ax=ax,
        xticklabels=['Benign', 'Malignant'],  # labels de l'axe x
        yticklabels=['Benign', 'Malignant']   # labels de l'axe y
    )
    ax.set_title(name, fontsize=10)

# Si la grille contient plus d'axes que de modèles (cases vides), on les désactive.
for idx in range(n_models, len(axes)):
    axes[idx].axis('off')

# Ajuste automatiquement les espacements pour éviter le chevauchement des titres/étiquettes.
plt.tight_layout()
plt.show()

# 9. Learning Curves

In this section we analyse how model performance evolves with training data and training epochs. Cell **9\*** uses `learning_curve` to show training and validation accuracy as a function of the number of training examples for the tuned models from Section 6, providing evidence about bias/variance and data efficiency. Cell **9.2** studies optimization dynamics for proxy SGD-based models, helping to diagnose overfitting and the stability of the training procedures.

In [ ]:
# =====================================================================
# 9*. Courbes d'apprentissage pour les modèles optimisés (Section 6)
# =====================================================================

from sklearn.model_selection import learning_curve, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score

# ---------------------------------------------------------------------
# Score personnalisé pour traiter la régression linéaire comme un classifieur
# ---------------------------------------------------------------------
# SGDRegressor renvoie des valeurs continues ; pour tracer une courbe
# d'apprentissage en "accuracy", on applique un seuil à 0,5 pour obtenir
# des prédictions binaires, puis on calcule l'accuracy classique.
def linear_reg_classifier_accuracy(y_true, y_pred_continuous):
    y_pred = (y_pred_continuous >= 0.5).astype(int)
    return accuracy_score(y_true, y_pred)

# make_scorer permet d'utiliser cette fonction comme scoring dans learning_curve
linear_acc_scorer = make_scorer(linear_reg_classifier_accuracy)


# ---------------------------------------------------------------------
# Fonction générique pour tracer une courbe d'apprentissage (train / validation)
# ---------------------------------------------------------------------
def plot_learning_curve(
    estimator,
    X,
    y,
    title,
    scoring,
    cv,
    ax,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5)
):
    # learning_curve entraîne le modèle sur des sous‑ensembles de plus en plus grands
    # de l'ensemble d'entraînement et renvoie les scores moyens en cross‑validation.
    train_sizes, train_scores, val_scores = learning_curve(
        estimator,
        X,
        y,
        train_sizes=train_sizes,
        cv=cv,
        scoring=scoring,
        n_jobs=n_jobs
    )

    # Moyenne et écart-type des scores (sur les folds de validation croisée)
    train_mean = train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)

    # Courbe de score d'entraînement
    ax.plot(train_sizes, train_mean, 'o-', label='Training', linewidth=2)
    ax.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15)

    # Courbe de score de validation
    ax.plot(train_sizes, val_mean, 'o-', label='Validation', linewidth=2)
    ax.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.15)

    ax.set_title(title)
    ax.set_xlabel('Nombre d\'exemples d\'entraînement')
    ax.set_ylabel('Accuracy')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best')


# ---------------------------------------------------------------------
# Recréation des modèles avec leurs hyperparamètres optimaux (Section 6)
# ---------------------------------------------------------------------
linear_est = SGDRegressor(
    loss='squared_error',
    learning_rate='constant',
    eta0=best_eta,
    max_iter=best_iter,
    random_state=42
)

softmax_est = LogisticRegression(
    C=best_softmax_C,
    solver=best_softmax_solver,
    max_iter=best_softmax_max_iter,
    random_state=42
)

mlp_est = MLPClassifier(
    hidden_layer_sizes=best_mlp_hidden,
    learning_rate_init=best_mlp_lr,
    learning_rate='adaptive',
    alpha=best_mlp_alpha,
    max_iter=best_mlp_it,
    batch_size=128,
    n_iter_no_change=20,
    early_stopping=True,
    random_state=42,
    validation_fraction=0.1
)

svm_est = SVC(
    C=best_svm_C,
    kernel='linear',
    probability=True,
    random_state=42
)

knn_l1_est = KNeighborsClassifier(
    n_neighbors=best_knn_l1_k,
    metric='minkowski',
    p=1
)

knn_l2_est = KNeighborsClassifier(
    n_neighbors=best_knn_l2_k,
    metric='minkowski',
    p=2
)

# StratifiedKFold : cross‑validation en conservant le ratio de classes dans chaque fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Liste des modèles pour lesquels on trace les courbes d'apprentissage.
# Pour la régression linéaire, on utilise le scorer personnalisé linear_acc_scorer.
models_lc = [
    ('Linear Regression', linear_est, linear_acc_scorer),
    ('Softmax Regression', softmax_est, 'accuracy'),
    ('MLP', mlp_est, 'accuracy'),
    ('L2-SVM', svm_est, 'accuracy'),
    ('L1-NN', knn_l1_est, 'accuracy'),
    ('L2-NN', knn_l2_est, 'accuracy')
]

n_models = len(models_lc)
fig, axes = plt.subplots(n_models, 1, figsize=(8, 4 * n_models))
if n_models == 1:
    axes = [axes]

# Boucle sur chaque modèle pour tracer la courbe train/validation
for ax, (name, est, scoring) in zip(axes, models_lc):
    plot_learning_curve(
        est,
        X_train_smote,
        y_train_smote,
        f'{name} Learning Curve (tuned model)',
        scoring=scoring,
        cv=cv,
        ax=ax,
        n_jobs=-1
    )

plt.tight_layout()
plt.show()

In [ ]:
# =====================================================================
# 9.2 Dynamique d'optimisation pour des modèles proxy basés sur SGD
# (et non les estimateurs finaux de la section 6)
# =====================================================================

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.model_selection import train_test_split

# ---------------------------------------------------------------------
# Fonction utilitaire : log-loss + accuracy pour un modèle probabiliste
# ---------------------------------------------------------------------
def compute_loss_acc(model, X, y):
    # Probabilités pour la classe positive
    proba = model.predict_proba(X)[:, 1]

    # Log-loss binaire implémentée manuellement
    loss = -np.mean(
        y * np.log(proba + 1e-10) +            # terme pour les positifs
        (1 - y) * np.log(1 - proba + 1e-10)    # terme pour les négatifs
    )

    # Prédictions de classe et accuracy correspondante
    pred = model.predict(X)
    acc = accuracy_score(y, pred)
    return loss, acc

# ---------------------------------------------------------------------
# Suivi de l'évolution de la perte/accuracy pour un MLP (proxy)
# ---------------------------------------------------------------------
def track_mlp_history(X_train, y_train, max_iter=300):
    # Re-split interne entraînement/validation pour le suivi des courbes
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )

    # MLP proxy de taille (50,50,50) entraîné itérativement avec warm_start
    model = MLPClassifier(
        hidden_layer_sizes=(50, 50, 50),
        learning_rate_init=0.005,
        alpha=0.05,
        max_iter=1,          # une seule itération par appel à fit
        random_state=42,
        warm_start=True      # conserve les poids entre les appels successifs à fit
    )

    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}

    # Boucle d'entraînement incrémental pour construire les courbes
    for _ in range(max_iter):
        model.fit(X_train_split, y_train_split)
        train_loss, train_acc = compute_loss_acc(model, X_train_split, y_train_split)
        val_loss, val_acc = compute_loss_acc(model, X_val, y_val)
        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)
    return history

# ---------------------------------------------------------------------
# Suivi de l'évolution de la perte/accuracy pour d'autres modèles SGD
# ---------------------------------------------------------------------
def track_model_history(model_class, loss_type, X_train, y_train, max_iter=300):
    # Re-split interne entraînement/validation
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )

    # Instanciation d'un modèle SGD (classifieur ou régression)
    model = model_class(
        loss=loss_type,              # type de perte : 'hinge', 'log_loss' ou 'squared_error'
        learning_rate='constant',
        eta0=1e-3,                   # petit taux d'apprentissage fixe
        max_iter=1,                  # une itération par fit
        random_state=42,
        warm_start=True              # conserve les poids entre les fits successifs
    )

    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}

    for _ in range(max_iter):
        model.fit(X_train_split, y_train_split)
        if hasattr(model, 'predict_proba'):
            # Cas des modèles probabilistes : on utilise compute_loss_acc
            train_loss, train_acc = compute_loss_acc(model, X_train_split, y_train_split)
            val_loss, val_acc = compute_loss_acc(model, X_val, y_val)
        else:
            # Cas des modèles sans predict_proba (par ex. régression ou SVM hinge)
            train_pred = model.predict(X_train_split)
            val_pred = model.predict(X_val)
            if loss_type == 'squared_error':
                # Perte quadratique moyenne pour la régression
                train_loss = np.mean((train_pred - y_train_split)**2)
                val_loss = np.mean((val_pred - y_val)**2)
                # Accuracy via seuillage à 0,5
                train_acc = accuracy_score(y_train_split, (train_pred >= 0.5).astype(int))
                val_acc = accuracy_score(y_val, (val_pred >= 0.5).astype(int))
            else:
                # Perte hinge pour un classifieur linéaire de type SVM
                decision_train = model.decision_function(X_train_split)
                decision_val = model.decision_function(X_val)
                train_loss = np.mean(np.maximum(0, 1 - y_train_split * decision_train))
                val_loss = np.mean(np.maximum(0, 1 - y_val * decision_val))
                train_acc = accuracy_score(y_train_split, train_pred)
                val_acc = accuracy_score(y_val, val_pred)
        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)
    return history

# ---------------------------------------------------------------------
# Construction des historiques pour différents modèles proxy
# ---------------------------------------------------------------------
all_models_history = {
    'MLP': track_mlp_history(X_train_smote, y_train_smote),
    'GRU-SVM': gru_model.history.history,  # historique Keras déjà calculé lors de l'entraînement du GRU
    'L2-SVM': track_model_history(SGDClassifier, 'hinge', X_train_smote, y_train_smote),
    'Softmax Regression': track_model_history(SGDClassifier, 'log_loss', X_train_smote, y_train_smote),
    'Linear Regression': track_model_history(SGDRegressor, 'squared_error', X_train_smote, y_train_smote)
}

# Analyse simple des gaps train/validation pour détecter du surapprentissage
for model_name, history in all_models_history.items():
    if history['loss'] and history['val_loss']:
        final_train_loss = history['loss'][-1]
        final_val_loss = history['val_loss'][-1]
        loss_gap = final_train_loss - final_val_loss
        final_train_acc = history['accuracy'][-1]
        final_val_acc = history['val_accuracy'][-1]
        acc_gap = final_train_acc - final_val_acc
        print(f"{model_name}:")
        print(f"  Loss Gap (Train - Val): {loss_gap:.4f}")
        print(f"  Accuracy Gap (Train - Val): {acc_gap:.4f}")
        if loss_gap > 0.1 or acc_gap > 0.1:
            print("  Possible overfitting detected.")
        else:
            print("  No significant overfitting.")
        print()

from sklearn.metrics import roc_auc_score, f1_score, classification_report

# ---------------------------------------------------------------------
# Évaluation complémentaire des modèles proxy sur le jeu de test
# (utile pour comparer leurs performances brutes)
# ---------------------------------------------------------------------
final_models = {
    'MLP': MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate_init=0.005, alpha=0.05, max_iter=1000, random_state=42),
    'GRU-SVM': gru_svm_model,
    'L2-SVM': SGDClassifier(loss='hinge', learning_rate='constant', eta0=1e-3, max_iter=1000, random_state=42),
    'Softmax Regression': SGDClassifier(loss='log_loss', learning_rate='constant', eta0=1e-3, max_iter=1000, random_state=42),
    'Linear Regression': SGDRegressor(loss='squared_error', learning_rate='constant', eta0=1e-3, max_iter=1000, random_state=42)
}

results = {}
for name, model in final_models.items():
    if name == 'GRU-SVM':
        # Pour GRU-SVM, on réutilise directement les prédictions déjà calculées
        y_pred = gru_svm_y_test_pred
        y_proba = gru_svm_y_test_proba
    else:
        model.fit(X_train_smote, y_train_smote)
        y_pred = model.predict(X_test_scaled)
        if hasattr(model, 'predict_proba'):
            y_proba = model.predict_proba(X_test_scaled)[:, 1]
        elif hasattr(model, 'decision_function'):
            y_proba = model.decision_function(X_test_scaled)
        else:
            y_proba = model.predict(X_test_scaled)
            y_pred = (y_proba >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    results[name] = {'AUC': auc, 'F1-Score': f1, 'Accuracy': acc}

for model, metrics in results.items():
    print(f"{model}:")
    print(f"  AUC: {metrics['AUC']:.3f}")
    print(f"  F1-Score: {metrics['F1-Score']:.3f}")
    print(f"  Accuracy: {metrics['Accuracy']:.3f}")
    print()

# 10. Conclusion

The experimental results show that all investigated models achieve high performance on this breast cancer detection task, with accuracies above 90% and ROC–AUC values close to 1.0. Among them, the tuned logistic regression (Softmax classifier) provides the best overall trade-off between accuracy and ROC–AUC on the test set, while simpler linear and k-NN baselines remain highly competitive. The confusion matrices confirm that false negatives are rare for the best models, and the learning curves suggest that, apart from the GRU–SVM hybrid, there is no severe overfitting. Overall, these findings support the use of well-regularized linear or shallow non-linear models as strong, interpretable baselines for breast cancer diagnosis on this dataset.

In [ ]:
best_model = results_df['accuracy'].idxmax()
best_accuracy = results_df.loc[best_model, 'accuracy']
best_roc_auc = results_df.loc[best_model, 'roc_auc']

print(f"Meilleur Modèle : {best_model}")
print(f"  Accuracy sur Test : {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"  ROC-AUC : {best_roc_auc:.4f}")

print("Tous les modèles ont dépassé le seuil de 90% d'accuracy :")
for model in results_df.index:
    acc = results_df.loc[model, 'accuracy']
    print(f"  {model}: {acc*100:.2f}%")